In [1]:
from keras import layers, models, losses, regularizers, initializers, optimizers, callbacks
from MyDataHandler import DataGenerator
import pandas as pd
import matplotlib.pyplot as plt

Setting up training and testing data sets

Partition index is used to decide the training/testing split. Currently set to 80/20

In [3]:
data = pd.read_csv('../data/chessData.csv')

parition_index = int( len(data) * 0.8 )

partitioned_IDs = {
    'training' : [i for i in range(0, 500000)],
    'testing' : [i for i in range(500000, 600000)]
}

params = {
    'data_frame' : data,
    'batch_size' : 128, 
    'x_dim' : (8,8,12),
    'y_dim' : (1,1),
    'output_type' : 'normalized'
}

training_data = DataGenerator(IDs = partitioned_IDs['training'], **params)
testing_data = DataGenerator( IDs = partitioned_IDs['testing'], **params)

In [4]:
print(data)

                                                        FEN Evaluation
0         rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR ...        -10
1         rnbqkbnr/pppp1ppp/4p3/8/4P3/8/PPPP1PPP/RNBQKBN...        +56
2         rnbqkbnr/pppp1ppp/4p3/8/3PP3/8/PPP2PPP/RNBQKBN...         -9
3         rnbqkbnr/ppp2ppp/4p3/3p4/3PP3/8/PPP2PPP/RNBQKB...        +52
4         rnbqkbnr/ppp2ppp/4p3/3p4/3PP3/8/PPPN1PPP/R1BQK...        -26
...                                                     ...        ...
12958030  r1bqkb1r/pp3ppp/1nn1p3/3pP3/3P1P2/1B3N2/PP2Q1P...         +6
12958031  r2qkb1r/pp1b1ppp/1nn1p3/3pP3/3P1P2/1B3N2/PP2Q1...        +84
12958032  r2qkb1r/pp1b1ppp/1nn1p3/3pP3/3P1P2/1BN2N2/PP2Q...          0
12958033  r2qkb1r/pp1b1ppp/1n2p3/n2pP3/3P1P2/1BN2N2/PP2Q...       +115
12958034  r2qkb1r/pp1b1ppp/1n2p3/n2pP3/3P1P2/2N2N2/PPB1Q...        +45

[12958035 rows x 2 columns]


Multilayer perceptron model - very similar paramters to the model used by Sabatelli et al.

In [5]:
model = models.Sequential()
model.add(layers.InputLayer( input_shape = (8,8,12)))
model.add(layers.Flatten())
model.add(layers.Dense(2048, activation = 'linear'))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(2048,activation= 'linear'))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(2048,activation= 'linear'))
model.add(layers.Dropout(0.2))
model.add(layers.Flatten())
model.add(layers.Dense(1,activation='sigmoid'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 768)               0         
                                                                 
 dense (Dense)               (None, 2048)              1574912   
                                                                 
 dropout (Dropout)           (None, 2048)              0         
                                                                 
 dense_1 (Dense)             (None, 2048)              4196352   
                                                                 
 dropout_1 (Dropout)         (None, 2048)              0         
                                                                 
 dense_2 (Dense)             (None, 1050)              2151450   
                                                                 
 dropout_2 (Dropout)         (None, 1050)              0

2022-12-19 18:57:18.607975: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Training

In [7]:
# training paramters used by Sabatelli et al.
# model.compile(
#     optimizer=optimizers.Adam(learning_rate=0.001, beta_1=0.90, beta_2=0.99, epsilon= 1.91828183), 
#     loss = losses.mean_squared_error, 
#     metrics=['mean_absolute_error'])

model.compile(optimizer = "adam", 
              loss = losses.MeanSquaredError(), 
              metrics=['mean_absolute_error']
              )
es = callbacks.EarlyStopping(monitor='loss', mode='min', verbose=1, patience = 5)

history = model.fit(training_data, 
                    epochs=50, 
                    validation_data=(testing_data), 
                    workers = 4,
                    use_multiprocessing=True,
                    callbacks= [es])

model.save('Model_12_12_18_22')

Epoch 1/50


Visualization

In [ ]:
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')

plt.show()